In [1]:
import os
from dotenv import load_dotenv

# get api address from .env file
load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_PRD")

In [2]:
from OptiHPLCHandler import EmpowerHandler

handler = EmpowerHandler(
    project="WebAPI_test",
    address=EMPOWER_API_ADDRESS,
    allow_login_without_context_manager=True,
)

handler.connection.verify = "../ca-certificates.crt"

# Get methods with BSM and QSM pumps as well as PDA and TUV detectors
with handler:
    bsm_pda_method = handler.GetInstrumentMethod("@BSM_PDA_Template")
    qsm_pda_method = handler.GetInstrumentMethod("@QSM_PDA_Template")
    bsm_tuv_method = handler.GetInstrumentMethod("@BSM_TUV_Template")
    qsm_pda_flr_method = handler.GetInstrumentMethod("@QSM_PDA_FLR_Template")
    # bsm_pda_rid_method = handler.GetInstrumentMethod("@BSM_PDA_RI_Template") # not implemented
    bsm_tuv_single_method = handler.GetInstrumentMethod("@BSM_TUV_Single_Template")
    bsm_tuv_dual_method = handler.GetInstrumentMethod("@BSM_TUV_Dual_Template")
    bsm_tuv_off_method = handler.GetInstrumentMethod("@BSM_TUV_Off_Template")
    bsm_pda_on_method = handler.GetInstrumentMethod("@BSM_PDA_ON_Template")

c:\Users\slcb\Desktop\Programming\OptiHPLCHandler\.env\lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'reappnnit024n.corp.novocorp.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
from OptiHPLCHandler.empower_detector_module_method import (
    PDAChannel,
    FLRChannel,
    TUVChannel,
    PDASpectralChannel,
)

# Channels
The channels in a method can be interacted with on two levels. On the instrument method level and on the detector level.
A PDA has space for 8 single wavelength channels and 1 spectral channel

### PDA Method

In [4]:
bsm_pda_method_copy = bsm_pda_method.copy()
print(bsm_pda_method_copy.channels)  # On instrument method level

[PDAChannel(wavelength1='222', wavelength2='498', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01')]


Changing channel using channels attribute (222 nm to 210 nm)

In [5]:
# Wavelength 1 is the key for a PDA channel, wavelength is the key for a TUV channel
# Not setting the other attributes will keep the default values defined by Empower
pda_channel = [PDAChannel(wavelength1=210)]
bsm_pda_method_copy.channels = pda_channel
print(bsm_pda_method_copy.channels)

[PDAChannel(wavelength1='210', wavelength2='254', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01')]


In [6]:
# One can set with another channel type that it is compatible with (e.g. TUVChannel)
# Useful for simple method conversion
tuv_channel = [TUVChannel(wavelength=666)]
bsm_pda_method_copy.channels = tuv_channel
print(bsm_pda_method_copy.channels)

[PDAChannel(wavelength1='666', wavelength2='254', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01')]


Changing channel using the wavelengths attribute (for convenience)

In [7]:
bsm_pda_method_copy.wavelengths = [210, 220, 230]  # 214 or "214"
print(bsm_pda_method_copy.wavelengths)

['210', '220', '230']


#### Spectral Channel
Only available on detector method level. Not the instrument method level

In [8]:
bsm_pda_on_method_copy = bsm_pda_on_method.copy()
bsm_pda_on_method_copy.channels  # On instrument method level

[PDAChannel(wavelength1='222', wavelength2='498', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01'),
 PDASpectralChannel(start_wavelength='210', end_wavelength='400', resolution='Resolution_12')]

In [9]:
bsm_pda_method_copy.detector_method_list[0].spectral_channel  # returns none is not set

In [10]:
bsm_pda_on_method_copy.detector_method_list[0].spectral_channel

PDASpectralChannel(start_wavelength='210', end_wavelength='400', resolution='Resolution_12')

In [11]:
spectral = PDASpectralChannel(
    start_wavelength=190,
    end_wavelength=400,
)
bsm_pda_on_method_copy.detector_method_list[0].spectral_channel = spectral
bsm_pda_on_method_copy.detector_method_list[0].spectral_channel

PDASpectralChannel(start_wavelength='190', end_wavelength='400', resolution='Resolution_12')

In [12]:
bsm_pda_on_method_copy.detector_method_list[0].spectral_wavelengths

[{'Start Wavelength': '190', 'End Wavelength': '400'}]

In [13]:
spectral = [{"Start Wavelength": "444", "End Wavelength": "555"}]
bsm_pda_on_method_copy.detector_method_list[0].spectral_wavelengths = spectral
bsm_pda_on_method_copy.detector_method_list[0].spectral_wavelengths

[{'Start Wavelength': '444', 'End Wavelength': '555'}]

### TUV Method
Operates the same as a PDA but only has space for two channels.
The detector method automatically handles the changing all the various enumerators for you when you switch from Single Mode to Dual Mode


In [14]:
bsm_tuv_method_copy = bsm_tuv_method.copy()
print(bsm_tuv_method_copy.channels)  # On instrument method level

[TUVChannel(wavelength='555', datarate='SingleDataRate_20A', datamode='SingleMode_1A', filtertype='Filter_2', timeconstant='0.1', ratiominimum='0.0001', autozerowavelength='Az_3', autozeroinjectstart=True, autozeroeventorkey=True)]


Changing channel using channels attribute (555 nm to 666 nm)

In [15]:
# One can set with another channel type that it is compatible with (e.g. TUVChannel)
# Useful for simple method conversion
tuv_channel = [TUVChannel(wavelength=666)]
bsm_tuv_method_copy.channels = tuv_channel
print(bsm_tuv_method_copy.channels)  # Note datarate etc for single mode

[TUVChannel(wavelength='666', datarate='SingleDataRate_20A', datamode='SingleMode_1A', filtertype='Filter_2', timeconstant='0.1000', ratiominimum='0.0001', autozerowavelength='Az_3', autozeroinjectstart=True, autozeroeventorkey=True)]


In [16]:
# Changing to dual mode
bsm_tuv_method_copy.channels = [TUVChannel(wavelength=111), TUVChannel(wavelength=222)]
print(
    bsm_tuv_method_copy.channels
)  # note different datarates, datamode and timeconstant

[TUVChannel(wavelength='111', datarate='DualDataRate_1B', datamode='DualModeA_1B', filtertype='Filter_2', timeconstant='2.0000', ratiominimum='0.0001', autozerowavelength='Az_3', autozeroinjectstart=True, autozeroeventorkey=True), TUVChannel(wavelength='222', datarate='DualDataRate_1B', datamode='DualModeB_2C', filtertype='Filter_2', timeconstant='2.0000', ratiominimum='0.0001', autozerowavelength='Az_3', autozeroinjectstart=True, autozeroeventorkey=True)]


### FLR Method

In [17]:
qsm_pda_flr_method_copy = qsm_pda_flr_method.copy()
channels = qsm_pda_flr_method_copy.channels
channels

[PDAChannel(wavelength1='214', wavelength2='798', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01'),
 PDAChannel(wavelength1='260', wavelength2='798', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01'),
 PDAChannel(wavelength1='280', wavelength2='798', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01'),
 FLRChannel(excitation='280', emission='348', channel_name='ChannelA', enable=True, datamode='Emission_1F')]

Set w/ channels attribute

In [18]:
# FLR channels can only be set with channels attribute and not wavelengths
channels = [FLRChannel(excitation=210, emission=220)]
channels  # Overwrites the PDA channel

[FLRChannel(excitation=210, emission=220, channel_name='', enable=True, datamode='Emission_1F')]

In [19]:
# How to change on channel level without overwriting the other channels
# alternatively, use wavelengths attribute (see below)

In [20]:
qsm_pda_flr_method_copy = qsm_pda_flr_method.copy()
channels = qsm_pda_flr_method_copy.channels
flr_channel = [FLRChannel(excitation=210, emission=220)]
pda_channels = channels[:-1]  # All but the last channel (FLR)
channels = pda_channels + flr_channel
channels

[PDAChannel(wavelength1='214', wavelength2='798', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01'),
 PDAChannel(wavelength1='260', wavelength2='798', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01'),
 PDAChannel(wavelength1='280', wavelength2='798', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01'),
 FLRChannel(excitation=210, emission=220, channel_name='', enable=True, datamode='Emission_1F')]

Set w/ wavelength attribute

In [21]:
qsm_pda_flr_method_copy = qsm_pda_flr_method.copy()
qsm_pda_flr_method_copy.wavelengths

['214',
 '260',
 '280',
 {'Excitation wavelength': '280', 'Emission wavelength': '348'}]

In [ ]:
qsm_pda_flr_method_copy.wavelengths = [210, 220, 230]
qsm_pda_flr_method_copy.wavelengths  # FLR cannot be set with wavelengths on the instrument method level

['210',
 '220',
 '230',
 {'Excitation wavelength': '270', 'Emission wavelength': '347'}]

In [23]:
qsm_pda_flr_method_copy.detector_method_list[1].wavelengths

[{'Excitation wavelength': '280', 'Emission wavelength': '348'}]

In [24]:
qsm_pda_flr_method_copy.detector_method_list[1].wavelengths = [
    {"Excitation wavelength": "270", "Emission wavelength": "347"}
]
qsm_pda_flr_method_copy.detector_method_list[1].wavelengths

[{'Excitation wavelength': '270', 'Emission wavelength': '347'}]